Gilad Shtern 31-Mar-2020

# Coronavirus in Israel

## Introduction

In this work, I attempt to analyze the Coronavirus in Israelby using a diffrent method. In my previous work I used the followed features to predict this virus: Country Geolocation, Continent cumulative cases, population density & human development index. The outcome was, that most features did not gave a good correlation with country cumulative new cases and deaths.

On this work, I attempt to predict the Coronavirus by using a comparison with the Flu epidemic as they have similarity.


## Table of Content:
Flu:
- Load Flu dataset from US CDC.
Data Exploration:
- Drop non-relevant columns
- Remove NAs.
- Convert columns format
- Adding week duration from first case - to align with Coronavirus
- Seasonal Flu graphs
- Find the most infected week

Coronavirus:
- Load Flu dataset from  ECDC.
- Use Israel relvant data
Data Exploration:
- Add week of year for future alignment between Corona to Flu datasets 
- Convert daily new cases into week cumulative dataset
- Adding week alignments
- Seasonal Flu  & Corona graphs


# Part 1 - Seasonal Flu

In [1]:
#Step:1 - Load Flu data
#Link into CDC Flu Dataset: https://apps.who.int/flumart/Default?ReportNo=12
import nbconvert
import pandas as pd
train_df = pd.read_excel("G:/DataScienceProject/CoronavirusIsrael/Influenza_Laboratory_Surveillance_Information_2017-20.xlsx")
train_df.drop(['Received/ collected', 'Influenza transmission zone', 'A (H5)', 'Total number of influenza negative viruses', 'ILI activity'],inplace=True,axis=1)
train_df.head(10)

,"Country, area or territory",WHO region,Year,Week,Start date,End date,Processed,A (H1),A (H1N1)pdm09,A (H3),A (not subtyped),A (Total),B (Yamagata lineage),B (Victoria lineage),B (lineage not determined),B (Total),Total number of influenza positive viruses
0,Israel,European Region of WHO,2017,31,2017-07-31,2017-08-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Israel,European Region of WHO,2017,32,2017-08-07,2017-08-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Israel,European Region of WHO,2017,33,2017-08-14,2017-08-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Israel,European Region of WHO,2017,34,2017-08-21,2017-08-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Israel,European Region of WHO,2017,35,2017-08-28,2017-09-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Israel,European Region of WHO,2017,36,2017-09-04,2017-09-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Israel,European Region of WHO,2017,37,2017-09-11,2017-09-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Israel,European Region of WHO,2017,38,2017-09-18,2017-09-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Israel,European Region of WHO,2017,39,2017-09-25,2017-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Israel,European Region of WHO,2017,40,2017-10-02,2017-10-08,120.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0


In [2]:
#Step-2: Drop NAs
train_df = train_df.dropna()
train_df.isna().sum()

Country, area or territory                    0
WHO region                                    0
Year                                          0
Week                                          0
Start date                                    0
End date                                      0
Processed                                     0
A (H1)                                        0
A (H1N1)pdm09                                 0
A (H3)                                        0
A (not subtyped)                              0
A (Total)                                     0
B (Yamagata lineage)                          0
B (Victoria lineage)                          0
B (lineage not determined)                    0
B (Total)                                     0
Total number of influenza positive viruses    0
dtype: int64

In [3]:
#Step-3: Convert numeric columns into int32 for ease future ML
colsList = list(train_df)
for i, value in enumerate(colsList):
    if train_df[value].dtypes == 'int64':
        train_df[value] = train_df[value].astype('int32')
    elif train_df[value].dtypes == 'float64':
        train_df[value] = train_df[value].astype('int32')

train_df.dtypes

Country, area or territory                            object
WHO region                                            object
Year                                                   int32
Week                                                   int32
Start date                                    datetime64[ns]
End date                                      datetime64[ns]
Processed                                              int32
A (H1)                                                 int32
A (H1N1)pdm09                                          int32
A (H3)                                                 int32
A (not subtyped)                                       int32
A (Total)                                              int32
B (Yamagata lineage)                                   int32
B (Victoria lineage)                                   int32
B (lineage not determined)                             int32
B (Total)                                              int32
Total number of influenz

In [4]:
train_df.head()

,"Country, area or territory",WHO region,Year,Week,Start date,End date,Processed,A (H1),A (H1N1)pdm09,A (H3),A (not subtyped),A (Total),B (Yamagata lineage),B (Victoria lineage),B (lineage not determined),B (Total),Total number of influenza positive viruses
9,Israel,European Region of WHO,2017,40,2017-10-02,2017-10-08,120,0,0,0,0,0,0,0,1,1,1
10,Israel,European Region of WHO,2017,41,2017-10-09,2017-10-15,109,0,0,0,0,0,0,0,2,2,2
11,Israel,European Region of WHO,2017,42,2017-10-16,2017-10-22,163,0,0,0,2,2,0,0,3,3,5
12,Israel,European Region of WHO,2017,43,2017-10-23,2017-10-29,132,0,0,0,0,0,0,0,1,1,1
13,Israel,European Region of WHO,2017,44,2017-10-30,2017-11-05,157,0,0,0,0,0,0,0,3,3,3


Now that, we cleansed the Flu dataset, we can compare between seasonal Flu by year.
Since, Coronaviruse is new type & as noted above the my previous features didn't had high correlation, we need a different approach.
In this approach, we will compare seasonal Flu - "Real Life" model, as baseline for analyzing Coronavirus.
In the below section, I'm comparing 3 Flu seasons: 2017-2018, 2018-2019, 2019-2020

In [5]:
#Step:4 - Seasonal Flu graphs
# Bokeh libraries
import warnings
warnings.filterwarnings('ignore')
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource

# Output to file
output_notebook()

# Isolate the data per years
flu_late_2017 = train_df[(train_df['Week'] > 39) & (train_df['Year'] == 2017)]
flu_early_2018 = train_df[(train_df['Week'] < 20) & (train_df['Year'] == 2018)]
frames = [flu_late_2017, flu_early_2018]
flu_2017_2018 = pd.concat(frames)

flu_late_2018 = train_df[(train_df['Week'] > 39) & (train_df['Year'] == 2018)]
flu_early_2019 = train_df[(train_df['Week'] < 20) & (train_df['Year'] == 2019)]
frames = [flu_late_2018, flu_early_2019]
flu_2018_2019 = pd.concat(frames)

flu_late_2019 = train_df[(train_df['Week'] > 39) & (train_df['Year'] == 2019)]
flu_early_2020 = train_df[(train_df['Week'] < 20) & (train_df['Year'] == 2020)]
frames = [flu_late_2019, flu_early_2020]
flu_2019_2020 = pd.concat(frames)

# Create a ColumnDataSource object for each team
flu_2017_2018_cases = ColumnDataSource(flu_2017_2018)
flu_2018_2019_cases = ColumnDataSource(flu_2018_2019)
flu_2019_2020_cases = ColumnDataSource(flu_2019_2020)

# Create and configure the figure
fig = figure(x_axis_type='datetime',
             plot_height=300, plot_width=900,
             title='New Cases',
             x_axis_label='Week', y_axis_label='Cases',
             toolbar_location=None)

# Render the race as step lines
fig.step('End date', 'Total number of influenza positive viruses', 
        color='#CE1141', legend='Flu_2017_2018', 
         source=flu_2017_2018_cases)
fig.step('End date', 'Total number of influenza positive viruses',
         color='#006BB6', legend='Flu_2018_2019', 
         source=flu_2018_2019_cases)
fig.step('End date', 'Total number of influenza positive viruses',
         color='#41CE11', legend='Flu_2019_2020', 
         source=flu_2019_2020_cases)

# Move the legend to the upper left corner
fig.legend.location = 'top_left'

# Show the plot
show(fig)

Loading BokehJS ...

The 2020 Flu curve is narrow, which mean this Flu infect fast with storng impact.
This may sugget two things: (A) Doctors were confiused between seasonal Flu to Corona, (B) A diferent type Flu earpted - Hence, may have same characteristics as Corona virus.

In [6]:
#Step:5 - Find the most infected week
maxInfec2020 = train_df['Total number of influenza positive viruses'].max()
maxInfectWeek2020 = train_df[train_df['Total number of influenza positive viruses'] == maxInfec2020]
print("Seasonal Flu 2020 - the highest infected ",  maxInfec2020 , " , where found on week num: ", str(maxInfectWeek2020['Week'])[0])

maxInfec2019 = flu_2018_2019['Total number of influenza positive viruses'].max()
maxInfectWeek2019 = flu_2018_2019[flu_2018_2019['Total number of influenza positive viruses'] == maxInfec2019]
print("Seasonal Flu 2019 - the highest infected ",  maxInfec2019 , " , where found on week num: ", str(maxInfectWeek2019['Week'])[0])

maxInfec2018 = flu_2017_2018['Total number of influenza positive viruses'].max()
maxInfectWeek2018 = flu_2017_2018[flu_2017_2018['Total number of influenza positive viruses'] == maxInfec2018]
print("Seasonal Flu 2018 - the highest infected ",  maxInfec2018 , " , where found on week num: ", str(maxInfectWeek2018['Week'])[0])

Seasonal Flu 2020 - the highest infected  308  , where found on week num:  1
Seasonal Flu 2019 - the highest infected  167  , where found on week num:  7
Seasonal Flu 2018 - the highest infected  155  , where found on week num:  2


# Part 2 - Corona Virus

In [7]:
#Step:6 - Load Coronavirus data
#Data was collected from ECDC: https://www.ecdc.europa.eu/en/publications-data/download-todays-data-geographic-distribution-covid-19-cases-worldwide
corona_df = pd.read_excel("G:/DataScienceProject/CoronavirusIsrael/COVID-19-geographic-disbtribution-worldwide-2020-03-30.xlsx")
corona_df = corona_df[corona_df['countriesAndTerritories'] == 'Israel']
corona_df.drop(['geoId', 'countryterritoryCode', 'popData2018', 'day', 'month', 'deaths'],inplace=True,axis=1)
corona_df.head()

,dateRep,year,cases,countriesAndTerritories
3653,2020-03-30,2020,628,Israel
3654,2020-03-29,2020,584,Israel
3655,2020-03-28,2020,369,Israel
3656,2020-03-27,2020,297,Israel
3657,2020-03-26,2020,439,Israel


In [8]:
#Step:7 - Add week of year for future alignment between Corona to Flu datasets 
corona_df['Week'] = corona_df['dateRep'].dt.week
corona_df.head(20)

,dateRep,year,cases,countriesAndTerritories,Week
3653,2020-03-30,2020,628,Israel,14
3654,2020-03-29,2020,584,Israel,13
3655,2020-03-28,2020,369,Israel,13
3656,2020-03-27,2020,297,Israel,13
3657,2020-03-26,2020,439,Israel,13
3658,2020-03-25,2020,488,Israel,13
3659,2020-03-24,2020,371,Israel,13
3660,2020-03-23,2020,188,Israel,13
3661,2020-03-22,2020,171,Israel,12
3662,2020-03-21,2020,35,Israel,12


In [9]:
#Step:8 - Convert daily new cases into week cumulative dataset
colList = list(corona_df)
weekList = list(corona_df['Week'].unique())
weekCorona_df = pd.DataFrame(columns=colList)
counter = 1
for i, value in enumerate(weekList):
    tmp = corona_df[corona_df['Week'] == value]
    weekCorona_df.loc[counter] = ['', 2020, tmp['cases'].sum(), 'Israel', value]
    counter += 1

weekCorona_df = weekCorona_df[weekCorona_df['cases'] > 0]
weekCorona_df

,dateRep,year,cases,countriesAndTerritories,Week
1,,2020,628,Israel,14
2,,2020,2736,Israel,13
3,,2020,705,Israel,12
4,,2020,153,Israel,11
5,,2020,18,Israel,10
6,,2020,6,Israel,9
7,,2020,1,Israel,8


In [10]:
#Step:9 - Adding week alignments
#Remove week with 0 infected
flu_2018_2019 = flu_2018_2019[flu_2018_2019['Total number of influenza positive viruses'] > 0]
flu_2019_2020 = flu_2019_2020[flu_2019_2020['Total number of influenza positive viruses'] > 0]
corona_df = corona_df[corona_df['Week'] > 0]

#Add new colmun for week alignment:
###corona_df['WeekAlign'] = ''

#Flu 2018 2019
flu_2018_2019['WeekAlign'] = ''
minYear = flu_2018_2019['Year'].min()
minWeek = flu_2018_2019[flu_2018_2019['Year'] == minYear]
minWeek = minWeek['Week'].min()
for i in range(len(flu_2018_2019)):
    if flu_2018_2019['Year'].iloc[i] == minYear:
        if flu_2018_2019['Week'].iloc[i] > 39:
            flu_2018_2019['WeekAlign'].iloc[i] = flu_2018_2019['Week'].iloc[i] - minWeek + 1
    else:
        flu_2018_2019['WeekAlign'].iloc[i] = flu_2018_2019['Week'].iloc[i] + 53 - minWeek

#Flu 2019 2020
flu_2019_2020['WeekAlign'] = ''
minYear = flu_2019_2020['Year'].min()
minWeek = flu_2019_2020[flu_2019_2020['Year'] == minYear]
minWeek = minWeek['Week'].min()
for i in range(len(flu_2019_2020)):
    if flu_2019_2020['Year'].iloc[i] == minYear:
        if flu_2019_2020['Week'].iloc[i] > 39:
            flu_2019_2020['WeekAlign'].iloc[i] = flu_2019_2020['Week'].iloc[i] - minWeek + 1
    else:
        flu_2019_2020['WeekAlign'].iloc[i] = flu_2019_2020['Week'].iloc[i] + 53 - minWeek
            
#Corona
#Flu 2019 2020
weekCorona_df['WeekAlign'] = ''
minWeek = weekCorona_df['Week'].min()
for i in range(len(weekCorona_df)):
            weekCorona_df['WeekAlign'].iloc[i] = weekCorona_df['Week'].iloc[i] - minWeek + 1

      

In [11]:
weekCorona_df

,dateRep,year,cases,countriesAndTerritories,Week,WeekAlign
1,,2020,628,Israel,14,7
2,,2020,2736,Israel,13,6
3,,2020,705,Israel,12,5
4,,2020,153,Israel,11,4
5,,2020,18,Israel,10,3
6,,2020,6,Israel,9,2
7,,2020,1,Israel,8,1


In [12]:
#Step:10 - Seasonal Flu  & Corona graphs
from bokeh.io import show
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.transform import dodge
from bokeh.io import output_notebook

# Output to file
output_notebook()

####
Flu2019_weeks = list(flu_2018_2019['WeekAlign'])
Flu2020_weeks = list(flu_2019_2020['WeekAlign'])
Corona_weeks = list(weekCorona_df['WeekAlign'])
week = set(Flu2019_weeks + Flu2020_weeks + Corona_weeks)
week = list(week)
weeklist = []
for i, value in enumerate(week):
    weeklist.append(str(value))

Corona_counts = list(weekCorona_df['cases'])[::-1]
Flu_2019_count = list(flu_2018_2019['Total number of influenza positive viruses'])
Flu_2020_count = list(flu_2019_2020['Total number of influenza positive viruses'])

epidemic = ['Corona', 'Flu_2019', 'Flu_2020']

data = {'weeklist' : weeklist,
        'Corona'   : Corona_counts,
        'Flu_2019'   : Flu_2019_count,
        'Flu_2020'   : Flu_2020_count}

source = ColumnDataSource(data=data)

p = figure(x_range=weeklist, y_range=(0, 4000), plot_height=250, title="Cases by Epeidemic",
           toolbar_location=None, tools="")

p.vbar(x=dodge('weeklist', -0.25, range=p.x_range), top='Corona', width=0.2, source=source,
       color="#c9d9d3", legend_label="Corona")

p.vbar(x=dodge('weeklist',  0.0,  range=p.x_range), top='Flu_2019', width=0.2, source=source,
       color="#718dbf", legend_label="Flu_2019")

p.vbar(x=dodge('weeklist',  0.25, range=p.x_range), top='Flu_2020', width=0.2, source=source,
       color="#e84d60", legend_label="Flu_2020")

p.xgrid.grid_line_color = None
p.y_range.start = 0

show(p)

Loading BokehJS ...

# Conclusion

The conclusion from this work is that Coronavirus is differed from well known Flu epidemic from years 2017-2020.
Futher, work is need to be able predict of Coronavirus impact.